<a href="https://colab.research.google.com/github/AriesAllen7/TF-IDF/blob/master/Red_pln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

referencia al proyecto que contiene VecText el cual es una libreria de Python para vectorizar texto en forma de matriz TF-IDF.

In [13]:
! git clone https://github.com/AriesAllen7/TF-IDF.git
import sys
sys.path.append('TF-IDF')
from VecText import VecText


fatal: destination path 'TF-IDF' already exists and is not an empty directory.


Todas las importaciones y declaraciones necesarias para el funcionamiento de la red y las funciones.

In [14]:
import numpy as np
import spacy
import re
import pandas as pd
import sklearn as sk
import math
import nltk
import tensorflow
from sklearn import preprocessing
nltk.download('brown')
from VecText import VecText
from nltk.corpus import brown
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
en_stops = set(stopwords.words('english'))
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from nltk import SnowballStemmer
englishstemmer=SnowballStemmer('english')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Creacion de los vectores con los documentos de las categorias pertenecientes al corpus de Brown con las que va a trabajar la red neuronal.

In [15]:
nlp = spacy.load('en_core_web_sm')
vector_madre_lore= brown.fileids(categories= 'lore')
vector_madre_learned= brown.fileids(categories= 'learned')
vector_madre_belles_letters= brown.fileids(categories= 'belles_lettres')
print (vector_madre_lore)
documentoL= [] ##vector donde se van a alojar todos los documentos.

['cf01', 'cf02', 'cf03', 'cf04', 'cf05', 'cf06', 'cf07', 'cf08', 'cf09', 'cf10', 'cf11', 'cf12', 'cf13', 'cf14', 'cf15', 'cf16', 'cf17', 'cf18', 'cf19', 'cf20', 'cf21', 'cf22', 'cf23', 'cf24', 'cf25', 'cf26', 'cf27', 'cf28', 'cf29', 'cf30', 'cf31', 'cf32', 'cf33', 'cf34', 'cf35', 'cf36', 'cf37', 'cf38', 'cf39', 'cf40', 'cf41', 'cf42', 'cf43', 'cf44', 'cf45', 'cf46', 'cf47', 'cf48']


creacion de las funciones patpara stematizar,lemantizar, normalizar y en general filtrar los documentos quitando signos, mayusculas, etc.

In [0]:
def STEM(texto):
    tokens = normalize(texto)  
    stems = [englishstemmer.stem(token) for token in tokens]
    stems = set(stems)
    strstem = ' '.join(stems)
    return strstem
def LEMMA(texto):
    doc = nlp(texto)
    lemmas = [tok.lemma_.lower() for tok in doc]
    lemmas = set(lemmas)
    strlem = ' '.join(lemmas)
    return strlem
def normalize(text):
    doc = nlp(text)
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and
    t.isalpha()]
    return lexical_tokens
def remover_signos(vocabulario):
    for x in re.findall("[''.,\/#!$?%\^&\*;:{}=\-_`~()”“\"…]", vocabulario):
        vocabulario = vocabulario.replace(x, " ")
    return vocabulario
def convertir_minusculas(vocabulario):
    for f in re.findall("([A-Z]+)", vocabulario):
        vocabulario = vocabulario.replace(f, f.lower())
    return vocabulario
def reduceTres(text):##opcional para remover palabras pequeñas
    doc = nlp(text)
    sstr = ""
    words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
    lexical_tokens = [t.lower() for t in words if len(t) > 3 and
    t.isalpha()]
    sstr = ' '.join(lexical_tokens)
    return sstr

En esta parte se combinan todos los documentos en un solo vector "documentoL" y a parte se filtra en diferentes ciclos for para lemantizar o stematizar implementando las funciones, ademas de remover las Stop Words

In [0]:
for x in range(len(vector_madre_lore)):
    aux= brown.words(fileids=vector_madre_lore[x])
    aux2=' '.join(aux)
    vocabulario= vocabulario+aux2
    documentoL.append(aux2)
for x in range(len(vector_madre_learned)):
    aux= brown.words(fileids=vector_madre_learned[x])
    aux2=' '.join(aux)
    vocabulario= vocabulario+aux2
    documentoL.append(aux2)
for x in range(len(vector_madre_belles_letters)):
    aux= brown.words(fileids=vector_madre_belles_letters[x])
    aux2=' '.join(aux)
    vocabulario= vocabulario+aux2
    documentoL.append(aux2)
 
copia = documentoL.copy()
for x in range(len(documentoL)):
    documentoL[x] = remover_signos(copia[x])
for x in range(len(documentoL)):
    documentoL[x] = convertir_minusculas(copia[x])
for x in range(len(documentoL)): ##aunque no hay una funcion especifica en este ciclo for se remueven las Stop Words.
    str = ""
    txt=[]
    for word in documentoL[x].split():
        if word not in en_stops:
            txt.append(word)
    str = ' '.join(txt)
    documentoL[x]=str
for x in range(len(documentoL)):
    documentoL[x]=reduceTres(documentoL[x])
for x in range(len(documentoL)):
    documentoL[x]=LEMMA(documentoL[x])
for x in range (len(documentoL)):
    documentoL[x]=STEM(documentoL[x])

Se vectoriza el texto en forma de matriz tf-idf y ademas aprovechando que estan en orden se agrega una etiqueta para la clasificacion de los documentos, se usa pandas para acoodar y mostrar.

In [18]:
VT = VecText(documentoL)
TF = VT.TF()
IDF = VT.IDF()
TF_IDF = VT.TF_IDF(TF, IDF)
vocabulary = VT.getVocabulary()
##etiquetas
vocabulary.append("categoria")
z = np.zeros([1,48])
b = np.ones([1,80])
c = []
for x in range(75):
    c.append(2)
w=np.concatenate((z,b), axis=None)
a=np.concatenate((w,c), axis=None)
TF_IDFN=(np.insert(TF_IDF, TF_IDF.shape[1], a, 1))
df = pd.DataFrame(TF_IDFN, columns=vocabulary)
print(df)
print(len(vocabulary))

     aaron   abandon  abaring  abas  ...  zoologist  zorrilla  zwei  categoria
0      0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        0.0
1      0.0  0.001818      0.0   0.0  ...        0.0       0.0   0.0        0.0
2      0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        0.0
3      0.0  0.001750      0.0   0.0  ...        0.0       0.0   0.0        0.0
4      0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        0.0
..     ...       ...      ...   ...  ...        ...       ...   ...        ...
198    0.0  0.001770      0.0   0.0  ...        0.0       0.0   0.0        2.0
199    0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        2.0
200    0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        2.0
201    0.0  0.001736      0.0   0.0  ...        0.0       0.0   0.0        2.0
202    0.0  0.000000      0.0   0.0  ...        0.0       0.0   0.0        2.0

[203 rows x 15339 columns]
15339


en esta parte se acomodan los valores en X y Y, ademas de transformar y clasificar de mejor manera los valores de Y.
se defie el modelo y en general el esquema de la red.
se imprimen los resultados y la marzi de confusion.

In [19]:
X = df.iloc[:, 1:(len(vocabulary)-1)].values
y = df.iloc[:, (len(vocabulary)-1)].values

encoder = LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

model = Sequential()

model.add(Dense(100, input_shape=((len(vocabulary)-2),), activation='sigmoid'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(Adam(lr=0.001), 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, epochs=200)
y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_test_class)
print(y_pred_class)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_class, y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 100)               1533800   
_________________________________________________________________
dense_20 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 153       
Total params: 1,539,003
Trainable params: 1,539,003
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
6/6 [==============================] - 0s 7ms/step - loss: 1.0499 - accuracy: 0.3642
Epoch 2/200
6/6 [==============================] - 0s 7ms/step - loss: 1.1718 - accuracy: 0.4012
Epoch 3/200
6/6 [==============================] - 0s 6ms/step - loss: 1.1618 - accuracy: 0.4012
Epoch 4/200
6/6 [==============================] - 0s 6ms/step - loss: 1.0759 - acc